In [38]:
from IPython.display import Image

- https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart
    - https://github.com/google-gemini/gemini-fullstack-langgraph-quickstart/blob/main/backend/src/agent/prompts.py
    - https://github.com/langchain-ai/react-agent/tree/main/src/react_agent
- 带着问题读源码
    - prompt templates
        - 学习 PE（Prompt Engineering）
        - instructions & ICE（In Context Example） & Output format
            - instructions：刻画的是约束
            - ice：In-Context learning
            - Output format：自动化整体的流程
    - deep research 是如何实现的；
        - agentic logic
        - 数据流/信息流：graph 中的各个 node 都在读取加工数据
    - 前后端全栈设计；
        - frontend: React, Vite
        - backend：langgraph-cli
    - 多模型的经济且有效的组合；
- complex search
    - `what are all the companies in the united states working on AI agents in 2025? make a list of at least 100. for each company, include the name, website, product, description of what they do, type of agents they build, and their vertical/industry.`

In [42]:
# img_data = graph.get_graph().draw_mermaid_png()
# with open('graph.png', 'wb') as f:
#     f.write(img_data)
Image(url='./figs/graph.png', width=100)

### full stack

- frontend/ ：包含使用 Vite 构建的 React 应用程序。
    - `http://localhost:5173/`
    - 技术选型
        - React: 一个由 Facebook 开发的库，用于构建用户界面。它的核心思想是把界面拆分成一个个独立、可复用的“组件”（Components）。
        - Vite: 一个现代化的前端构建工具和开发服务器。它的特点是“快”，能让你在修改代码后立刻看到浏览器中的变化。
        - TypeScript: JavaScript 的一个超集，增加了静态类型检查。如果你熟悉 Python 的类型提示（Type Hinting），可以把它理解为前端的类型提示，能帮助我们更早地发现错误。
    - `App.tsx`

```typescript
let initial_search_query_count = 0;
let max_research_loops = 0;
switch (effort) {
  case "low":
    initial_search_query_count = 1;
    max_research_loops = 1;
    break;
  case "medium":
    initial_search_query_count = 3;
    max_research_loops = 3;
    break;
  case "high":
    initial_search_query_count = 5;
    max_research_loops = 10;
    break;
}

...


thread.submit({
   messages: newMessages,
   initial_search_query_count: initial_search_query_count,
   max_research_loops: max_research_loops,
   reasoning_model: model,
});
```

- backend/ ：包含 LangGraph/FastAPI 应用程序，包括 research agent logic。
    - langgraph：state machine
- 启动 `make dev`（`Makefile` recipes）

```mermaid
sequenceDiagram
    participant User as 用户
    participant Browser as 浏览器 (前端 @ 5173)
    participant Backend as 后端服务 (Agent @ 2024)

    User->>+Browser: 1. 打开 http://localhost:5173
    Browser->>+Backend: 2. useStream hook 建立连接到 http://localhost:2024
    User->>Browser: 3. 输入问题并提交
    Browser->>Backend: 4. 发送包含问题的 API 请求 (thread.submit)
    activate Backend
    Note over Backend: Agent 开始处理...<br/>(生成查询, 网络搜索, 反思)
    Backend-->>Browser: 5. [流式] 返回处理过程中的事件
    deactivate Backend
    Browser->>User: 6. 实时更新UI，展示Agent活动
    activate Backend
    Backend-->>Browser: 7. [流式] 返回最终答案
    deactivate Backend
    Browser->>User: 8. 展示最终答案
```

### 模型组合

- 前端表单
    ```python
    const [effort, setEffort] = useState("medium");
    const [model, setModel] = useState("gemini-2.5-flash-preview-04-17");
    ```
    - effort
```tsx
switch (effort) {
    case "low":
      initial_search_query_count = 1;
      max_research_loops = 1;
      break;
    case "medium":
      initial_search_query_count = 3;
      max_research_loops = 3;
      break;
    case "high":
      initial_search_query_count = 5;
      max_research_loops = 10;
      break;
}
```
    - model
        - gemini-2.0-flash
        - gemini-2.5-flash-preview-04-17
        - gemini-2.5-pro-preview-05-06
- 后端
    - query_generator_model：gemini-2.0-flash
    - reflection_model：gemini-2.5-flash-preview-04-17
    - answer_model：gemini-2.5-pro-preview-05-06

### query_writer_instructions

- user (complex) query => single sub-queries
- format output
    ```python
    - Format your response as a JSON object with ALL three of these exact keys:
       - "rationale": Brief explanation of why these queries are relevant
       - "query": A list of search queries
    ```
- ice (in-context eamples)
```
Topic: What revenue grew more last year apple stock or the number of people buying an iphone
```json
{{
    "rationale": "To answer this comparative growth question accurately, we need specific data points on Apple's stock performance and iPhone sales metrics. These queries target the precise financial information needed: company revenue trends, product-specific unit sales figures, and stock price movement over the same fiscal period for direct comparison.",
    "query": ["Apple total revenue growth fiscal year 2024", "iPhone unit sales growth fiscal year 2024", "Apple stock price growth fiscal year 2024"],
}}
```
```

### langgraph

### tools

```python
response = genai_client.models.generate_content(
    model=configurable.query_generator_model,
    contents=formatted_prompt,
    config={
        "tools": [{"google_search": {}}],
        "temperature": 0,
    },
)
```

In [5]:
# !pip install google-genai
# !pip install langchain-google-genai

In [7]:
from google.genai import Client
from langchain_google_genai import ChatGoogleGenerativeAI

In [27]:
from dotenv import load_dotenv
assert load_dotenv()

In [9]:
genai_client = Client()

In [10]:
web_searcher_instructions = """Conduct targeted Google Searches to gather the most recent, credible information on "{research_topic}" and synthesize it into a verifiable text artifact.

Instructions:
- Query should ensure that the most current information is gathered. The current date is {current_date}.
- Conduct multiple, diverse searches to gather comprehensive information.
- Consolidate key findings while meticulously tracking the source(s) for each specific piece of information.
- The output should be a well-written summary or report based on your search findings. 
- Only include the information found in the search results, don't make up any information.

Research Topic:
{research_topic}
"""

In [12]:
from datetime import datetime
def get_current_date():
    return datetime.now().strftime("%B %d, %Y")

In [13]:
formatted_prompt = web_searcher_instructions.format(
    current_date=get_current_date(),
    research_topic="今年高考新闻",
)

In [14]:
# Uses the google genai client as the langchain client doesn't return grounding metadata
response = genai_client.models.generate_content(
    model='gemini-2.0-flash',
    contents=formatted_prompt,
    config={
        "tools": [{"google_search": {}}],
        "temperature": 0,
    },
)

In [31]:
# response.text: 报告正文
# response.candidates[0].grounding_metadata: 
# response.candidates[0].grounding_metadata.grounding_chunks: 存着所有找到的网页来源
# response.candidates[0].grounding_metadata.grounding_supports

In [32]:
response.candidates[0].grounding_metadata.grounding_chunks[0]

GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(domain=None, title='xuancheng.gov.cn', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUZIYQHsXRtCzlcdd_viRJgWBj6T4TDIDSUn9UXw0RG_hM2clvcQRanFubcwUOHKuYfHVfrK46OBAoiId6TnrfXFCqE4ppJC8RSHVoRkhUD_oTI93YED5IrM-Io2V4juutIxXTXw_5-OA6e8_emgz9B-tQ=='))

In [33]:
response.candidates[0].grounding_metadata.grounding_supports[0]

GroundingSupport(confidence_scores=None, grounding_chunk_indices=[0, 1, 2, 3, 4, 5, 6], segment=Segment(end_index=311, part_index=None, start_index=307, text='**1.'))

In [35]:
len(response.candidates[0].grounding_metadata.grounding_chunks), len(response.candidates[0].grounding_metadata.grounding_supports)

(18, 64)

In [36]:
import json

from IPython.display import display, HTML, Markdown


def show_json(obj):
  print(json.dumps(obj.model_dump(exclude_none=True), indent=2))

def show_parts(r):
  parts = r.candidates[0].content.parts
  if parts is None:
    finish_reason = r.candidates[0].finish_reason
    print(f'{finish_reason=}')
    return
  for part in r.candidates[0].content.parts:
    if part.text:
      display(Markdown(part.text))
    elif part.executable_code:
      display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    else:
      show_json(part)

  grounding_metadata = r.candidates[0].grounding_metadata
  if grounding_metadata and grounding_metadata.search_entry_point:
    display(HTML(grounding_metadata.search_entry_point.rendered_content))

In [37]:
show_parts(response)

Okay, I will conduct targeted Google Searches to gather the most recent, credible information on "今年高考新闻" (This year's Gaokao news) and synthesize it into a verifiable text artifact.


Here's a summary of the 2025 Gaokao (National College Entrance Examination) news, based on the search results:

**1. Gaokao Completion and Key Dates:**

*   The 2025 Gaokao has concluded in most provinces by June 10th. (2025-06-09, 2025-06-10, 2025-06-12, 2025-06-10)
*   Score release dates are concentrated around June 23rd to 26th, with specific provinces like Tianjin (June 24th), Inner Mongolia (June 24th), Guangxi (June 25th afternoon), Sichuan (June 25th evening), Hebei (June 25th), Zhejiang (June 26th), and Shandong (June 26th) having projected dates. (2025-06-11, 2025-06-09, 2025-06-09)
*   Several provinces have announced the start dates for志愿填报 (zhìyuàn tiánbào - volunteer filling/ application): Henan (June 26th), Hunan (June 26th), Shanxi (June 27th), Guizhou (June 28th-July 2nd), and Shandong (June 29th). (2025-06-10, 2025-06-14, 2025-06-09)
*   Beijing's first志愿填报 is scheduled from June 27th to July 1st. (2025-06-09)

**2. New Gaokao Reforms:**

*   **New高考体系基本建立 (xīn gāokǎo tǐxì jīběn jiànlì - New Gaokao system basically established):** A new Gaokao system is being implemented in more regions. By 2025, 29 provinces have adopted the new Gaokao system. (2025-06-04)
*   **"3+1+2" Model:** Sichuan, Henan, Shanxi, Inner Mongolia, Yunnan, Shaanxi, Qinghai and Ningxia are implementing the "3+1+2" model for the first time in 2025. This includes three unified national exam subjects (Chinese, Mathematics, Foreign Language), one compulsory subject chosen from Physics or History, and two elective subjects chosen from Chemistry, Geography, Politics, and Biology. (2025-06-04)
*   **批次合并 (pīcì hébìng - Batch Mergers):** 取消区分一二本，合并为本科批次 (qǔxiāo qūfēn yī'èrběn, hébìng wèi běnkē pīcì - Cancel the distinction between the first and second batches, and merge them into the undergraduate batch). (2025-03-11)
*   **志愿数量剧增 (zhìyuàn shùliàng jùzēng - The number of volunteers has increased sharply):** 新高考实行"专业+院校"的平行志愿模式 (xīn gāokǎo shíxíng "zhuānyè+yuànxiào" de píngxíng zhìyuàn móshì - The new Gaokao implements a parallel volunteer model of "major + university"). (2025-03-11)
*   **等级赋分 (děngjí fùfēn - Grade assignment):** In some provinces, some subjects are graded using a "等级赋分" system where scores are converted to a grade. (2025-06-04)
*   **投档录取 (tóudàng lùqǔ - Filing and admission):** 考生电子档案由投档到院校改为投档到院校专业组 (kǎoshēng diànzǐ dàng'àn yóu tóudàng dào yuànxiào gǎi wèi tóudàng dào yuànxiào zhuānyè zǔ - The electronic files of candidates are changed from being submitted to the university to being submitted to the university's major group). (2025-06-04)

**3. 志愿填报 Considerations and Guidance:**

*   Familiarize yourself with provincial Gaokao policies, gather information from reliable sources (official websites, 高考高招类报纸 (gāokǎo gāozhāo lèi bàozhǐ - Gaokao enrollment newspapers), etc.), and do a "preliminary selection" of志愿填报. (2025-06-14)
*   模拟填报 (mónǐ tiánbào - Mock filling): It is important to do a "模拟填报" to avoid mistakes during the actual process. (2025-06-14)
*   Consider factors such as your scores, the universities, and their programs. (2025-06-14)
*   阳光高考信息平台 (yángguāng gāokǎo xìnxī píngtái - Sunshine Gaokao Information Platform) is an important resource. (2025-06-14)
*   教育部 (jiàoyù bù - Ministry of Education) will conduct online and offline志愿填报 consulting activities. (2025-06-09, 2025-05-28)
*   Be aware of the new志愿填报 methods under the new Gaokao system. (2025-05-08)
*   Pay attention to the选科要求 (xuǎn kē yāoqiú - subject selection requirements) for志愿填报. (2025-03-11)
*   高校招生章程 (gāoxiào zhāoshēng zhāngchéng - University Admissions Regulations) are an important source of information. (2025-05-27)

**4. Key Policy Changes & Information:**

*   公立学校 (gōnglì xuéxiào - Public schools) are prohibited from recruiting 高考复读生 (gāokǎo fùdúshēng - Gaokao repeaters). (2025-03-11)
*   More foreign language options are available for the 外语科目 (wàiyǔ kēmù - foreign language subject). (2025-03-11)
*   综合素质 (zōnghé sùzhì - Comprehensive quality) is included in the enrollment reference conditions. (2025-03-11)
*   打击高考移民行为 (dǎjí gāokǎo yímín xíngwéi - Crackdown on Gaokao immigration). (2025-03-11)
*   Some universities are expanding their招生省份范围 (zhāoshēng shěngfèn fànwéi - enrollment province scope). (2025-06-10)
*   多所高校新增专业 (duō suǒ gāoxiào xīn zēng zhuānyè - Many universities have added new majors) or 招生项目 (zhāoshēng xiàngmù - enrollment projects). (2025-06-10)
*   多所高校表示将放松学生转专业的限制 (duō suǒ gāoxiào biǎoshì jiāng fàngsōng xuéshēng zhuǎn zhuānyè de xiànzhì - Many universities have stated that they will relax restrictions on students changing majors). (2025-06-10)
*   外语听力 (wàiyǔ tīnglì - Foreign language listening) will be tested once in June, instead of twice in January. (2024-09-19)

**5. Important Reminders and Cautions:**

*   Be wary of scams related to提前查分 (tíqián cháfēn - checking scores in advance) and 补录名额 (bǔlù míng'é - supplementary enrollment quotas). (2025-06-09, 2025-06-10)
*   Protect your personal information and be aware of potential information leaks. (2025-06-10, 2025-06-19)
*   Check official channels for information. (2025-06-19)

This information should provide a solid overview of the key news and updates related to the 2025 Gaokao.


### citation

```python
response = genai_client.models.generate_content(
    model=configurable.query_generator_model,
    contents=formatted_prompt,
    config={
        "tools": [{"google_search": {}}],
        "temperature": 0,
    },
)
```
- `response`
    - `response.text`: 模型生成的原始文本回答。
    - `response.candidates[0].grounding_metadata`: "溯源元数据"，这是实现引用的关键。它详细记录了文本中的哪一段话 (segment) 是基于哪个搜索结果 (grounding_chunks) 生成的。
    - resolve_urls
        - { "原始长链接": "生成的短链接" }。

## frontend

- App.tsx => main.tsx
    - TypeScript + JSX
    - TypeScript 是 JavaScript 的一个“超集”。可以把它想象成是“带有类型检查的 JavaScript”。

```
frontend/
├── public/              # 存放静态资源，如网站图标 vite.svg
├── src/
│   ├── components/      # React 组件
│   │   ├── ui/          # 从 shadcn/ui 来的基础 UI 组件
│   │   ├── ActivityTimeline.tsx  # 显示代理思考过程的时间线组件
│   │   ├── ChatMessagesView.tsx  # 聊天消息展示组件
│   │   ├── InputForm.tsx         # 用户输入表单组件
│   │   └── WelcomeScreen.tsx     # 欢迎界面组件
│   │
│   ├── lib/             # 工具函数
│   │   └── utils.ts     # cn() 函数，用于合并 Tailwind CSS 类名
│   │
│   ├── App.tsx          # 应用主组件，组织了所有其他组件
│   ├── main.tsx         # 应用入口文件，将 App 组件渲染到页面上
│   └── global.css       # 全局样式和 Tailwind CSS 配置
│
├── package.json         # 项目依赖和脚本配置
├── vite.config.ts       # Vite 配置文件，比如配置代理服务器
└── tsconfig.json        # TypeScript 配置文件
```


```mermaid
graph TD
    subgraph "Application Entry"
        A["main.tsx"] --> B["App.tsx"]
    end

    subgraph "Main Application Logic (App.tsx)"
        B --> C{"messages.length === 0 ?"}
        C -- Yes --> D["WelcomeScreen"]
        C -- No --> E["ChatMessagesView"]
    end

    subgraph "Welcome Screen"
        D --> F["InputForm"]
    end

    subgraph "Chat View"
        E --> G["HumanMessageBubble"]
        E --> H["AiMessageBubble"]
        E --> F
        H --> I["ActivityTimeline"]
    end

    subgraph "Shared & UI Components"
        F --> J["ui/Button, ui/Textarea, etc."]
        I --> K["ui/Card, etc."]
        G --> K
        H --> K
    end

    style B fill:#f9f,stroke:#333,stroke-width:2px
    style F fill:#ccf,stroke:#333,stroke-width:2px
    style I fill:#cfc,stroke:#333,stroke-width:2px
    style E fill:#fcf,stroke:#333,stroke-width:2px
```